### Label encoding 

Es una codificación ordinal simple: asigna un número entero único a cada categoría.

In [1]:
import pandas as pd

df = pd.DataFrame({
    'color': ['rojo', 'verde', 'azul', 'rojo', 'azul']
})

df['color_label'] = df['color'].astype('category').cat.codes
print(df)


   color  color_label
0   rojo            1
1  verde            2
2   azul            0
3   rojo            1
4   azul            0


#### Problema
Introduce orden artificial: el modelo puede pensar que rojo > verde > azul, lo cual no tiene sentido en variables categóricas nominales.



### Target encoding
Reemplaza cada categoría por el valor promedio del target para esa categoría.




In [2]:
df = pd.DataFrame({
    'color': ['rojo', 'verde', 'azul', 'rojo', 'azul'],
    'y':     [1,        0,        1,       0,      1]
})

df.groupby('color')['y'].mean()


color
azul     1.0
rojo     0.5
verde    0.0
Name: y, dtype: float64

### Problema 
Puede causar overfitting, especialmente cuando hay pocas observaciones por categoría (ej. categorías raras).



## Aplicando al titanic 


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool

# Cargar Titanic desde seaborn (puede usar también Kaggle)
df = pd.read_csv('Data/titanic.csv')
# Limpiar un poco
df = df[['Survived', 'Sex', 'Embarked']].dropna()

# Codificamos solo "embarked" para este ejemplo
X = df[['Embarked']]
y = df['Survived']

### Label encoding

In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X_label = X.copy()
X_label['Embarked'] = le.fit_transform(X_label['Embarked'])

X_label.head()


,Embarked
0,2
1,0
2,2
3,2
4,2


### Target encoding

In [7]:
mean_target = df.groupby('Embarked')['Survived'].mean()
X_target = X.copy()
X_target['Embarked'] = X_target['Embarked'].map(mean_target)

X_target.head()


,Embarked
0,0.336957
1,0.553571
2,0.336957
3,0.336957
4,0.336957


## Catboost encoding

#### ¿Cómo CatBoost maneja variables categóricas?

CatBoost puede trabajar directamente con variables categóricas **sin necesidad de encoding manual** (como one-hot o label encoding).

Esto lo logra utilizando una **forma especial de target encoding regularizado**, diseñada para evitar **overfitting** y **data leakage**.

### 1. Orden aleatorio + acumulación (sin usar el target actual)
Durante el entrenamiento, CatBoost:

- Ordena las filas de forma aleatoria.

- Para cada fila, codifica la categoría con el promedio acumulado del target de las observaciones anteriores con esa misma categoría.

- Nunca incluye el target de la fila actual en el cálculo →  no hay data leakage.

Esto ya funciona como una forma de regularización natural: las primeras apariciones tienen poca información y por lo tanto un encoding más neutro (por ejemplo, cero o el promedio global).

### 2. Múltiples permutaciones del dataset
CatBoost no hace esto una sola vez, sino que:

Genera varias permutaciones aleatorias del dataset.

Calcula el encoding acumulado en cada una.

Y luego promedia los resultados.

_Esto hace el encoding más estable y reduce la varianza_

### 3. Regularización tipo bayesiana
Además, CatBoost aplica una suavización adicional, similar al smoothing bayesiano:

$$ encoding = \frac{\sum y  + a * \mu}{n + a} $$

Donde:

- ∑y = suma de los targets anteriores para esa categoría
- n = número de ocurrencias anteriores
- μ = promedio global del target
- a = parámetro de suavizamiento




Esto suaviza los valores extremos y evita que categorías raras generen codificaciones inestables.



In [8]:
import pandas as pd

# Suponemos que ya cargaste tu DataFrame con columnas 'Embarked' y 'Survived'

# Mezclamos aleatoriamente el orden de las filas (simula una permutación interna de CatBoost)
df = df.sample(frac=1).reset_index(drop=True)

# Parámetros para suavizamiento
alpha = 10  # entre más alto, más peso al promedio global
global_mean = df['Survived'].mean()  # promedio global del target

# Inicializamos la columna de codificación y el diccionario de stats por categoría
df['catboost_enc'] = 0.0
enc_stats = {}  # {'Embarked_value': [suma_y, conteo]}

# Recorremos las filas en orden aleatorio
for i, row in df.iterrows():
    cat = row['Embarked']  # categoría actual
    
    # Si ya tenemos historial para esa categoría, aplicamos codificación suavizada
    if cat in enc_stats:
        sum_y, count_y = enc_stats[cat]
        enc_val = (sum_y + alpha * global_mean) / (count_y + alpha)
    else:
        # Si es la primera vez que aparece la categoría, usamos el promedio global
        enc_val = global_mean
    
    # Asignamos el encoding a la fila actual
    df.at[i, 'catboost_enc'] = enc_val
    
    # Actualizamos el historial con el valor actual (después de usarlo para codificar)
    if cat not in enc_stats:
        enc_stats[cat] = [row['Survived'], 1]
    else:
        enc_stats[cat][0] += row['Survived']
        enc_stats[cat][1] += 1

# Mostrar las primeras 20 filas para inspeccionar cómo evoluciona el encoding
df[['Survived', 'Embarked', 'catboost_enc']].head(10)


,Survived,Embarked,catboost_enc
0,0,S,0.382452
1,0,S,0.347684
2,1,C,0.382452
3,0,S,0.318710
4,0,S,0.294194
5,1,S,0.273180
6,1,C,0.438593
7,0,S,0.321635
8,0,S,0.301533
9,1,S,0.283795
